## Сортировка событий

* Пусть есть некоторые отрезки во времени (например, отрезок когда человек находился на сайте) 
* Что-то интересное происходит только в те моменты, когда человек приходит или уходит - события 
* Надо что-нибудь посчитать 

### Задача №1 

Сайт посетило $N$ человек, для каждого известно время входа на сайт $In_i$ и время выхода с сайта $Out_i$. Считается, что человек был на сайте с момента $In_i$ по $Out_i$ включительно. Определите, какое максимальное кол-во человек было на сайте одновременно.  

`Решение:`

Каждое событие здесь - пара, в которое первое число время, а второе - тип события. Сначала будем сортировать по времени, а потом по типу, причем событие $In$ должно быть раньше $Out$.

In [ ]:
def maxvisitorsonline(n, tin, tout):
    events = []
    # идем по людям
    for i in range(n):
        # добавляем время + тип события (-1 - пришел, 1 - ушел)
        events.append((tin[i], -1))
        events.append((tout[i], 1))
    # сортируем все события по времени
    events.sort()
    online = 0 
    maxonline = 0
    # идем по событиям и если чел пришел, то добавляем в кол-во людей онлайн одного
    # если чел ушел, то удаляем его из счетчика кол-ва людей онлайн
    for event in events:
        if event[1] == -1:
            online +=1 
        else: 
            online -= 1
        maxonline = max(online, maxonline)
    return maxonline

`Сложность по времени:` $O(N + N\log N)$  
`Сложность по памяти:` $O(N)$

### Задача №2

Сайт посетило $N$ человек, для каждого известно время входа на сайт $In_i$ и время выхода с сайта $Out_i$. Считается, что человек был на сайте с момента $In_i$ по $Out_i$ включительно. Определите, какое суммарное время на сайте был хотя бы один человек. 

`Решение:`

Если мы пришли в событие с положительным счетчиком кол-ва людей, то между этим и предыдущим событием на сайте кто-то был. Прибавим к ответу время между текущим и предыдущим событием. 

In [2]:
def timeonline(n, tin, tout):
    events = []
    # идем по людям
    for i in range(n):
        # добавляем время + тип события (-1 - пришел, 1 - ушел)
        events.append((tin[i], -1))
        events.append((tout[i], 1))
    # сортируем все события по времени
    events.sort()
    online = 0 
    time = 0
    # теперь идем не по событиям, а по их индексам, чтобы узнать про предыдущее событие
    for i in range(len(events)):
        # если > 0 людей (перед тем как обработали текущее), то значит люди уже были
        # поэтому считаем время между последним и предпоследним событием
        if online > 0:
            time += events[i][0] - events[i - 1][0]
        # а теперь обрабатываем число людей 
        if events[i][1] == -1:
            online += 1
        else: 
            online -= 1
    return time

`Сложность по времени:` $O(N + N\log N)$  
`Сложность по памяти:` $O(N)$

### Задача №3

Сайт посетило $N$ человек, для каждого известно время входа на сайт $In_i$ и время выхода с сайта $Out_i$. Считается, что человек был на сайте с момента $In_i$ по $Out_i$ включительно. Теперь начальник заходил на саайт $M$ раз в моменты времени $Boss_i$ и смотрел, сколько людей сейчас онлайн. Посещения сайта начальником упорядочены по времени. Определите, какие показания счетчика людей онлайн увидел начальник. 

`Решение:`

Создадим третий тип событий - вход начальника. При наступлении такого события будем сохранять текущее значение счетчика посетителей. 

In [ ]:
def bosscounters(n, tin, tout, m, tboss):
    events = []
    # идем по людям
    for i in range(n):
        # добавляем время + тип события (-1 - пришел, 1 - ушел)
        events.append((tin[i], -1))
        events.append((tout[i], 1))
    # добавляем еще m событий прихода начальника
    for i in range(m):
        events.append((tboss[i], 0))
    # сортируем все события по времени
    events.sort()
    online = 0 
    bossans = []
    for i in range(len(events)):
        if events[i][1] == -1:
            online += 1
        elif events[i][1] == 1: 
            online -= 1
        else:
            bossans.append(online)
    return bossans

`Сложность по времени:` $O(N + M + (N+M)\log(N+M))$  
`Сложность по памяти:` $O(N+M)$

## Два прохода

### Задача №4 

>На парковке в торговом центре $N$ мест. За день в ТЦ приезжало $M$ автомобилей, при этом некоторые из них длинные и занимали несколько подряд идущих парковочных мест. Для каждого автомобиля известно время приезда и отъезда, а также два числа - с какого по какое парковочные места он занимал. Если в какой-то момент времени один автомобиль уехал с парковочного места, то место считается освободившимся и в тот же момент времени на его место может встать другой. 

Необходимо определить, был ли момент, в который были заняты все парковочные места. 

`Решение:`

События - приезд и отъезд автомобиля (причем отъезд должен происходить раньше). Пудем поддерживать кол-во занятых мест и если после очередного события счетчик равен $N$, то такие моменты были

In [ ]:
def isparkingfull(cars, n):
    events = []
    for car in cars: 
        # машина это время приезда, отъезда и с какого по какое место было занято
        timein, timeout, placefrom, placeto = car
        # добавляем событие приезда и то какие места были заняты
        events.append((timein, 1, placeto-placefrom + 1))
        # добавляем событие отъезда и то какие места освободились
        events.append((timeout, -1, placeto-placefrom + 1))
    # сортируем события
    events.sort()
    occupied = 0 # сколько мест на парковке занято
    for i in range(len(events)):
        # если машина уехала, то места освобождаются
        if events[i][1] == -1:
            occupied -= events[i][2]
        # если машина приехала то места занимаются
        elif events[i][1] == 1:
            occupied += events[i][2]
        # если кол-во занятых мест == n, то парковка заполнена
        if occupied == n:
            return True
    return False

### Задача №5

>На парковке в торговом центре $N$ мест. За день в ТЦ приезжало $M$ автомобилей, при этом некоторые из них длинные и занимали несколько подряд идущих парковочных мест. Для каждого автомобиля известно время приезда и отъезда, а также два числа - с какого по какое парковочные места он занимал. Если в какой-то момент времени один автомобиль уехал с парковочного места, то место считается освободившимся и в тот же момент времени на его место может встать другой. 

Необходимо определить, был ли момент, в который были заняты все парковочные места и определить минимальное кол-во автомобилей, которое заняло все места. Если такого момента не было - вернуть $M+1$

`Решение:`

Добавим еще один счетчик на кол-во автомобилей и будем обновлять минимальное кол-во автомобилей, когда заняты все места

In [ ]:
def mincarsonfullparking(cars, n):
    events = []
    for car in cars: 
        # машина это время приезда, отъезда и с какого по какое место было занято
        timein, timeout, placefrom, placeto = car
        # добавляем событие приезда и то какие места были заняты
        events.append((timein, 1, placeto-placefrom + 1))
        # добавляем событие отъезда и то какие места освободились
        events.append((timeout, -1, placeto-placefrom + 1))
    # сортируем события
    events.sort()
    occupied = 0 # сколько мест на парковке занято
    nowcars = 0 # еще один счетчик, отвечающий за кол-во машин
    mincars = len(cars) + 1 # минимальное число машин на парковке достигалось, чтоб забить ее полностью
    for i in range(len(events)):
        # если машина уехала, то места освобождаются
        if events[i][1] == -1:
            occupied -= events[i][2]
            nowcars -= 1
        # если машина приехала то места занимаются
        elif events[i][1] == 1:
            occupied += events[i][2]
            nowcars += 1
        # если кол-во занятых мест == n, то парковка заполнена
        if occupied == n:
            mincars = min(mincars, nowcars)
    return mincars

### Задача №6

>На парковке в торговом центре $N$ мест. За день в ТЦ приезжало $M$ автомобилей, при этом некоторые из них длинные и занимали несколько подряд идущих парковочных мест. Для каждого автомобиля известно время приезда и отъезда, а также два числа - с какого по какое парковочные места он занимал. Если в какой-то момент времени один автомобиль уехал с парковочного места, то место считается освободившимся и в тот же момент времени на его место может встать другой. 

Необходимо определить, был ли момент, в который были заняты все парковочные места и определить минимальное кол-во автомобилей, которое заняло все места, а также номера этих автомобилей (в том порядке, как они перечисляются в списке). Если парковка никогда не была занята, то полностью вернуть пустой список

`Решение:`

Добавим в событие номер автомобиля в списке. При обновлении минимума просто копируем текущее состояние в ответ. Например, можно класть номера машин в множество либо в булевый список (где индекс - номер машины и True будет стоять, если машина с этим номером стоит). Множество лучше, если машины часто приезжают ненадолго, а список - если машины приезжают надолго. 

In [ ]:
def mincarsonfullparking(cars, n):
    events = []
    # теперь уже бежим по номерам машин, чтоб сохранять это
    for i in range(len(cars)): 
        # машина это время приезда, отъезда и с какого по какое место было занято
        timein, timeout, placefrom, placeto = cars[i]
        # добавляем событие приезда и то какие места были заняты + номер машины
        events.append((timein, 1, placeto-placefrom + 1, i))
        # добавляем событие отъезда и то какие места освободились + номер машины
        events.append((timeout, -1, placeto-placefrom + 1, i))
    # сортируем события
    events.sort()
    occupied = 0 # сколько мест на парковке занято
    nowcars = 0 # еще один счетчик, отвечающий за кол-во машин на парковке
    mincars = len(cars) + 1 # минимальное число машин на парковке достигалось, чтоб забить ее полностью
    carnums = set() # номера текущих машин (через множество)
    bestcarnums = set() # лучшие номера машин
    for i in range(len(events)):
        if events[i][1] == -1:
            occupied -= events[i][2]
            nowcars -= 1
            carnums.remove(events[i][3])
        # если машина приехала то места занимаются
        elif events[i][1] == 1:
            occupied += events[i][2]
            nowcars += 1
            carnums.add(events[i][3])
        if occupied == n and nowcars < mincars:
            bestcarnums = carnums.copy()
            mincars = nowcars
    return bestcarnums

Решение неэффективное из-за copy(). В среднем оно занимает M/2. Случится это может на каждые 3 события. То есть в итоге получаем $\frac{2M}{3} \cdot \frac{M}{2} = O(M^2)$. Рассмотрим эффективное решение **через 2 прохода**:

In [ ]:
def mincarsonfullparking(cars, n):
    events = []
    for i in range(len(cars)): 
        timein, timeout, placefrom, placeto = cars[i]
        events.append((timein, 1, placeto-placefrom + 1, i))
        events.append((timeout, -1, placeto-placefrom + 1, i))
    events.sort()
    occupied = 0
    nowcars = 0
    mincars = len(cars) + 1
    # 1-ый проход: ищем минимальное число машин
    for i in range(len(events)):
        if events[i][1] == -1:
            occupied -= events[i][2]
            nowcars -= 1
        elif events[i][1] == 1:
            occupied += events[i][2]
            nowcars += 1
        if occupied == n and nowcars < mincars:
            mincars = nowcars
    # 2-ой проход: ищем номера, уже зная оптимальный минимум на число машин
    carnums = set()
    nowcars = 0 
    for i in range(len(events)):
        if events[i][1] == -1:
            occupied -= events[i][2]
            nowcars -= 1
            carnums.remove(events[i][3])
        elif events[i][1] == 1:
            occupied += events[i][2]
            nowcars += 1
            carnums.add(events[i][3])
        if occupied == n and nowcars < mincars:
            return carnums
    return set()